<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/Pandas/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2%D1%8B%D0%B3%D1%80%D1%83%D0%B7%D0%BE%D0%BA_1%D0%A1_%D0%B8_%D0%A1%D0%91%D0%98%D0%A1/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2%D1%8B%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B8_%D0%A1%D0%91%D0%98%D0%A1_%D0%B8_1%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import glob
from google.colab import files as colab_files
import io,os
from datetime import datetime

# СБИС
# Загрузка файлов csv из СБИС в единый датафрейм df
pd.set_option('display.max_columns',40)
files=glob.glob('Входящие*.csv')
dfs=[]
for file in files:
  df_small=pd.read_csv(file, encoding='windows-1251', delimiter=';')
  dfs.append(df_small)
df=pd.concat(dfs,ignore_index=True) # Объединение данных по столбцам, создание новых последовательных индексов
df.columns=[col.replace(' ', '_').replace('.', '') for col in df.columns] # Переименование столбцов

# 1C
# Загрузка файла csv из 1С по-одному с папки вручную в датафрейм df2
uploaded=colab_files.upload()
filename=list(uploaded.keys())[0]
content=uploaded[filename]
df2=pd.read_csv(io.BytesIO(content), encoding='windows-1251', delimiter=';')
if os.path.exists(filename):
    os.remove(filename) # Удаление временного файла

# Добавление новых столбцов: "Номер счет-фактуры","Сумма счет-фактуры","Дата счет-фактуры","Сравнение дат"
for col in ['Номер счет-фактуры', 'Сумма счет-фактуры', 'Дата счет-фактуры', 'Сравнение дат']:
  df2[col] = None
# Если "Поставщик" - ЕАПТЕКА, то к "Номер накладной" нужно добавить /15 (унифицирование данных СБИС и 1С)
if 'Поставщик' in df2.columns and 'Номер накладной' in df2.columns:
  mask=df2['Поставщик'].str.startswith('ЕАПТЕКА', na=False) # NaN значения становятся False
  df2.loc[mask,'Номер накладной']=df2.loc[mask,'Номер накладной'].astype(str) + '/15'

# из СБИС в 1С
# Выгрузка записей в выгрузке из СБИСа по номеру накладной ".../15" в датафрейм df3
if 'Номер' in df.columns:
  mask=df2['Номер накладной'].str.endswith('/15', na=False) # Поиск строк в 1С
  l=df2.loc[mask, 'Номер накладной'].tolist() # Формирование списка номеров накладных из 1C
  df3=df[df['Номер'].isin(l)] # Датафрейм из СБИС с найденными накладными

# Выгрузка записей в выгрузке из СБИСа df3 по типу документа в датафрейм df4
if 'Тип_документа' in df3.columns:
  l2=['СчФктр','УпдДоп','УпдСчфДоп','ЭДОНакл']
  df4=df3[df3['Тип_документа'].isin(l2)] # Выбор документов по типу
  df4=df4.drop_duplicates(subset=['Номер'], keep='first') # Выбор первой записи
  df4['Дата'] = pd.to_datetime(df4['Дата'],format='mixed',dayfirst=True,errors='coerce').dt.strftime('%d.%m.%Y')
  df2['Дата накладной']=pd.to_datetime(df2['Дата накладной'], dayfirst=True, errors='coerce').dt.strftime('%d.%m.%Y')
  df2['Сумма счет-фактуры']=df2['Номер накладной'].map(df4.set_index('Номер')['Сумма']) # Добавление данных о сфет-фактуре
  df2['Дата счет-фактуры']=df2['Номер накладной'].map(df4.set_index('Номер')['Дата'])
  mask_both=df2['Дата счет-фактуры'].notna() & df2['Дата накладной'].notna()  # Сравнение дат накладной и счета-фактуры и заполнение столбца
  df2.loc[mask_both, 'Сравнение дат']=""
  df2.loc[mask_both & (df2['Дата счет-фактуры'] != df2['Дата накладной']), 'Сравнение дат']="Не совпадает!"
# Список с финальной сортировкой столбцов
final=['№ п/п','Штрих-код партии','Наименование товара','Поставщик','Дата приходного документа',
       'Номер приходного документа','Дата накладной','Номер накладной','Номер счет-фактуры',
       'Сумма счет-фактуры','Кол-во','Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
       'Сумма НДС','Сумма в закупочных ценах с НДС','Дата счет-фактуры','Сравнение дат']
df_result=df2[[col for col in final if col in df2.columns]] # Отсортированный результирующий датафрейм

# Выгрузка датафрейма в файл
date_folder=datetime.now().strftime("%Y-%m-%d")
result_path=os.path.join('Результат', date_folder)
os.makedirs(result_path, exist_ok=True) # Полный путь
base_name=os.path.splitext(filename)[0] # Имя загруженного файла по аптеке без расширения
output_file=os.path.join(result_path, f'{base_name}_результат.xlsx')
df_result.to_excel(output_file, sheet_name=base_name[:31], index=False)  # Обрезка названия до 31 символа - предел для названия листа в Excel

df2[df2['Сравнение дат']=='Не совпадает!']


Saving А123.csv to А123 (1).csv


,№ п/п,Штрих-код партии,Наименование товара,Поставщик,Дата приходного документа,Номер приходного документа,Дата накладной,Номер накладной,Кол-во,Сумма в закупочных ценах без НДС,Ставка НДС поставщика,Сумма НДС,Сумма в закупочных ценах с НДС,Номер счет-фактуры,Сумма счет-фактуры,Дата счет-фактуры,Сравнение дат
2789,2790,201300056838,"АНГИДАК 0,255МГ/ДОЗА 30МЛ. СПРЕЙ Д/МЕСТ.ПРИМ. ...",ЕАПТЕКА ООО,21.09.2021,13003391.0,21.09.2021,16411131/15,1.0,273.4,10%,27.34,300.74,None,"822,36",22.09.2021,Не совпадает!
2790,2791,201300056839,ГРАММИДИН НЕО С АНЕСТЕТИКОМ №18 ТАБ. Д/РАСС.,ЕАПТЕКА ООО,21.09.2021,13003391.0,21.09.2021,16411131/15,1.0,283.9,10%,28.39,312.29,None,"822,36",22.09.2021,Не совпадает!
2791,2792,201300056840,"РИНОСТОП ЭКСТРА 0,025% 10МЛ. НАЗАЛ.КАПЛИ ФЛ.",ЕАПТЕКА ООО,21.09.2021,13003391.0,21.09.2021,16411131/15,1.0,112.0,10%,11.20,123.20,None,"822,36",22.09.2021,Не совпадает!
2792,2793,201300056841,АЦИКЛОВИР-АКРИХИН 5% 5Г. №1 МАЗЬ Д/НАРУЖ.ПРИМ....,ЕАПТЕКА ООО,21.09.2021,13003391.0,21.09.2021,16411131/15,1.0,28.9,10%,2.89,31.79,None,"822,36",22.09.2021,Не совпадает!
2793,2794,201300056842,АЦИКЛОВИР-АКРИХИН 200МГ. №20 ТАБ. /АКРИХИН/,ЕАПТЕКА ООО,21.09.2021,13003391.0,21.09.2021,16411131/15,1.0,49.4,10%,4.94,54.34,None,"822,36",22.09.2021,Не совпадает!
